In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import sklearn

## 데이터 로드 및 train test로 분할

In [ ]:
# df = pd.read_csv('38_Oasis.csv').drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link','target'])

df = pd.read_csv('38_Oasis.csv').drop(columns = ['Unnamed: 0']).reset_index().drop(columns=['index','link','target'])

In [ ]:
df.head()

## null 값 확인

In [ ]:
df.info()

In [ ]:
# NULL 값 있는 행 확인
df[df['content'].isnull()]

In [ ]:
# 공백으로 NULL 값 채우기
df = df.fillna('')

In [ ]:
df.info()

내용에 빈 값이 있지만 title에서 추출할 것이 있지 않을까 해서 공백으로 채워줌

## index 날짜로 바꿔주기

In [ ]:
# date를 오름차순 정렬
df.date = df.date.sort_index(ascending=True)
# index로 설정
df = df.set_index('date')
df

## 특수문자 제거 정규표현식

In [ ]:
# 특수문자 제거 함수

import re
 
def cleanText(readData):
 
    #텍스트에 포함되어 있는 특수 문자 제거
 
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', readData)
 
    return text


In [ ]:
# df['title'] = df['title'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')
# df['content'] = df['content'].str.replace(r"[^\uAC00-\uD7A30-9a-zA-Z\s]",'')

In [ ]:
df['cc'] = '//'

In [ ]:
df['content'] = df['content']+df['cc']

In [ ]:
df

In [ ]:
df = df.drop(columns=['cc'])

In [ ]:
df = df.groupby('date').sum()
df

In [ ]:
df['label'] = '-'

In [ ]:
df

In [ ]:
df.to_csv('38_label.csv')

## 토큰화

In [ ]:
okt = Okt()


### 추출했던 word_list에서 불용어들만 정리

In [ ]:
import csv

with open("stopword.csv", 'w') as file:
    writer = csv.writer(file)
    writer.writerow(word_list)

In [ ]:
with open('stopword_add.csv', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    data = list(reader)

In [ ]:
data

In [ ]:
stopword_add = sum(data, [])
stopword_add

In [ ]:
# 불용어 처리 

stopwords = stopword_add + ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','\n','\n ', '제목']
stopwords

In [ ]:
title = []
for sentence in tqdm(df['title']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_title = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    title.append(stopwords_removed_title)

In [ ]:
print(len(title))

In [ ]:
from collections import Counter

In [ ]:
# 한 날짜? 한 행당 횟수 추출

for i in range(136):
    vocab = Counter(title[i])
    print(vocab)


In [ ]:
# 전체 중 빈도수 출력

all_word_list_title = sum(title, [])
vocab = Counter(all_word_list_title)
print(vocab)

In [ ]:
word_list = list(vocab.keys())
word_list

In [ ]:
word_rank = list(vocab.values())
word_rank

In [ ]:
data = {'word': word_list, 'count' : word_rank}
word_data = pd.DataFrame.from_dict(data)
word_data

In [ ]:
word_data = word_data.sort_values(by=['count'], ascending=False).reset_index().drop(columns=['index'])

In [ ]:
for i in range(26):
    print(word_data[ i*10 : (i+1)*10 ])

word_data.to_csv('sentimental(38).csv')

In [ ]:
word_data

sentimental = pd.read_csv('sentimental(38).csv').drop(columns=['Unnamed: 0'])
sentimental = sentimental.fillna('-')

sentimental

sentimental.to_csv('sentimental(38).csv')

In [ ]:
community_title_label = pd.read_csv('sentimental(38)_label_title.csv').drop(columns=['Unnamed: 0'])
community_title_label

### labeling 한 거에서 추가 불용어 처리

In [ ]:
community_title_label = community_title_label[~community_title_label['label'].str.contains('--')]
community_title_label

In [ ]:
community_title_label.to_csv('community_title_label.csv')

In [ ]:
data['label'].value_counts().plot(kind = 'bar')